in this notebook i checked what was downloaded by download_tickers_history.py.

i check that i have active and inactive tickers for all days when exchanges were open.

In [ ]:
# setup client for AWS S3
import sys
import os
# Add the parent directory to Python path to import api_key module
sys.path.append(os.path.dirname(os.path.abspath('')))
import settings

In [ ]:
tickers_dir = os.path.join(settings.ABSOLUTE_DATA_DIR, 'tickers')
active_tickers_file = os.path.join(tickers_dir, 'tickers_history_active.csv')
inactive_tickers_file = os.path.join(tickers_dir, 'tickers_history_inactive.csv')

In [ ]:
import csv

def collect_unique_dates(file_path):
    unique_dates = set()
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        next(reader, None)  # skip the first line (header)
        prev_date = None
        line_counter = 0
        for row in reader:
            if row:  # skip empty lines
                date = row[0]
                # optimization: check against the hashset only if the date is different from the previous one
                if date != prev_date:
                    unique_dates.add(date)
                    prev_date = date
            else:
                print(f"Empty line found in {file_path} at position {line_counter}, skipping.")
                return None
            line_counter += 1
    return unique_dates

In [ ]:
import pandas as pd

dates = collect_unique_dates(active_tickers_file)
df = pd.DataFrame(list(dates), columns=['Date'])
df

In [ ]:
# sort the DataFrame by date
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date').reset_index(drop=True)
# save the DataFrame to a CSV file
df.to_csv(os.path.join(tickers_dir, 'unique_active_dates.csv'), index=False)

In [ ]:
import pandas_market_calendars as mcal

nyse = mcal.get_calendar('NYSE')

schedule = nyse.schedule(start_date='2003-09-10', end_date='2025-06-11')
# get dates from the schedule
schedule_dates = pd.Series(schedule.index.date)
schedule_dates

In [ ]:
# convert df['Date'] to datetime
active_dates = pd.Series(pd.to_datetime(df['Date']).dt.date)
active_dates

In [ ]:
# check that unique_days are in the NYSE schedule
print(schedule_dates.isin(active_dates).all())
# and that the NYSE schedule is in unique_days
print(active_dates.isin(schedule_dates).all())


this True, False result is expected because active_dates were downloaded for all days except weekends. means that it has dates for federal holidays and also days of disasters when exchanges were closed. one more check below.

In [ ]:
# Find dates that are in schedule_dates but not in active_dates
missing_in_active = schedule_dates[~schedule_dates.isin(active_dates)]
print("Dates in NYSE schedule but not in active dates:")
print(missing_in_active)

# Find dates that are in active_dates but not in schedule_dates
extra_in_active = active_dates[~active_dates.isin(schedule_dates)]
print("Dates in active dates but not in NYSE schedule:")
print(extra_in_active)

In [ ]:
# now do the same for inactive tickers
inactive_dates = collect_unique_dates(inactive_tickers_file)

df = pd.DataFrame(list(inactive_dates), columns=['Date'])
df

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date').reset_index(drop=True)
# save the DataFrame to a CSV file
df.to_csv(os.path.join(tickers_dir, 'unique_inactive_dates.csv'), index=False)

In [ ]:
inactive_dates = pd.Series(pd.to_datetime(df['Date']).dt.date)
inactive_dates

In [ ]:
# check that unique_days are in the NYSE schedule
print(schedule_dates.isin(active_dates).all())
# and that the NYSE schedule is in unique_days
print(active_dates.isin(schedule_dates).all())

ok. so i have active and inactive tickers for all days when exchanges were open. next step is to try to calculate renames.